# alpha

In [1]:
import pandas as pd

from zipline import optimize as opt

from zipline.research import prices

In [2]:
# 尽量选择同时包含正负收益的股票列表
ps = prices(['000001','000002','000333','600645','600771'],'2018-5-19','2018-5-29')

[2018-06-05 08:32:04.860263] INFO: zipline.finance.metrics.tracker: 模拟1个交易日
首个开盘时间: 2018-05-29 01:31:00+00:00
最后收盘时间: 2018-05-29 07:00:00+00:00


In [3]:
alphas = ps.pct_change(6).iloc[-1]

## 修改数据以利更好观察

In [4]:
alphas = pd.Series([-0.03,-0.10,-0.,0.03,0.11],index=alphas.index)

In [5]:
alphas

平安银行(000001)   -0.03
万 科Ａ(000002)   -0.10
美的集团(000333)   -0.00
中源协和(600645)    0.03
广誉远(600771)     0.11
dtype: float64

In [6]:
_long, _short = alphas.index[4], alphas.index[1]

# 优化

## 期初投资组合

In [7]:
current_portfolio = pd.Series([-0.2,-0.1,0.0001,0.1,0.2], index=alphas.index)

In [8]:
current_portfolio

平安银行(000001)   -0.2000
万 科Ａ(000002)   -0.1000
美的集团(000333)    0.0001
中源协和(600645)    0.1000
广誉远(600771)     0.2000
dtype: float64

## 最大alpha

In [9]:
objective = opt.MaximizeAlpha(alphas)

In [10]:
constraints = [
    #opt.MaxGrossExposure(3.0),
    #opt.NetExposure(-0.2,0.2),
    #opt.DollarNeutral(),
    opt.Pair(_long, _short, 2),
    #opt.Basket(current_portfolio.index[:3],-0.3,-0.1),
    #opt.Frozen(current_portfolio.index[2]),
    #opt.ReduceOnly(current_portfolio.index[:3]),
    #opt.LongOnly(current_portfolio.index[:2]),
    opt.NotLessThan(0.1),
    opt.NotExceed(0.5),
    #opt.FixedWeight(current_portfolio.index[2],0.3),
    #opt.Basket(current_portfolio.index[:3],-0.3,-0.1),
]

In [11]:
result = opt.run_optimization(objective, constraints, current_portfolio)

# 不可行约束

In [12]:
result.prob.status

'infeasible'

In [13]:
result.print_diagnostics()

没有找到满足所有必需约束的投资组合，尝试优化失败。检查以下投资组合，发现违背约束条件：
万 科Ａ(000002)不可能同时满足<Pair(广誉远(600771)，万 科Ａ(000002)，对冲比率：2)>和<NotLessThan(权重>=0.1)>约束
万 科Ａ(000002)不可能同时满足<Pair(广誉远(600771)，万 科Ａ(000002)，对冲比率：2)>和<NotExceed(区间[-0.5,+0.5])>约束
广誉远(600771)不可能同时满足<Pair(广誉远(600771)，万 科Ａ(000002)，对冲比率：2)>和<NotExceed(区间[-0.5,+0.5])>约束



In [14]:
result.new_weights

# 无界

In [15]:
constraints = [
    #opt.MaxGrossExposure(3.0),
    #opt.NetExposure(-0.2,0.2),
    #opt.DollarNeutral(),
    opt.Pair(_long, _short, 2),
    #opt.Basket(current_portfolio.index[:3],-0.3,-0.1),
    #opt.Frozen(current_portfolio.index[2]),
    #opt.ReduceOnly(current_portfolio.index[:3]),
    #opt.LongOnly(current_portfolio.index[:2]),
    opt.NotLessThan(0.1),
    #opt.NotExceed(0.5),
    #opt.FixedWeight(current_portfolio.index[2],0.3),
    #opt.Basket(current_portfolio.index[:3],-0.3,-0.1),
]

In [16]:
result = opt.run_optimization(objective, constraints, current_portfolio)

In [17]:
result.prob.status

'unbounded'

In [18]:
result.print_diagnostics()

问题无界。如没有限制总权重，求解最大alpha时，目标权重无上界


# 修改约束

In [19]:
constraints = [
    opt.MaxGrossExposure(3.0),
    #opt.NetExposure(-0.2,0.2),
    #opt.DollarNeutral(),
    opt.Pair(_long, _short, 2),
    #opt.Basket(current_portfolio.index[:3],-0.3,-0.1),
    #opt.Frozen(current_portfolio.index[2]),
    #opt.ReduceOnly(current_portfolio.index[:3]),
    #opt.LongOnly(current_portfolio.index[:2]),
#     opt.NotLessThan(0.1),
    #opt.NotExceed(0.5),
    #opt.FixedWeight(current_portfolio.index[2],0.3),
    #opt.Basket(current_portfolio.index[:3],-0.3,-0.1),
]

In [20]:
result = opt.run_optimization(objective, constraints, current_portfolio)

In [21]:
result.new_weights

平安银行(000001)    0.0
万 科Ａ(000002)   -1.0
美的集团(000333)    0.0
中源协和(600645)   -0.0
广誉远(600771)     2.0
dtype: float64